# 图卷积神经网络

- 将使用 PyTorch Geometric (PyG) 框架学习 GCN，应用于图分类任务。 

## 图分类


- 图分类是指在给定图数据集的情况下，根据某些结构图属性对整个图进行分类的问题。
- 图分类最常见的任务是分子属性预测，其中分子被表示为图，任务可能是推断分子是否抑制HIV病毒复制。

- 多特蒙德工业大学收集了各种不同的图形分类数据集，称为 TUDatasets，可以通过 PyTorch Geometric 中的 torch_geometric.datasets.TUDataset 访问。 让我们加载并检查较小的数据集之一，即 MUTAG 数据集

In [2]:
import torch
from torch_geometric.datasets import TUDataset  #引入数据集

dataset = TUDataset(root='data/TUDataset', name='MUTAG') # 加载数据集

Processing...
Done!


In [3]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')


Dataset: MUTAG(188):
Number of graphs: 188
Number of features: 7
Number of classes: 2


In [4]:
data = dataset[0] # 得到数据中的第一个图

print()
print(data)
print('=============================================================')


Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])


In [5]:
# 获得图的一些统计特征
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of nodes: 17
Number of edges: 38
Average node degree: 2.24
Has isolated nodes: False
Has self-loops: False
Is undirected: True


-  根据以上，我们可以看出该数据集提供了 188 个不同的图，现在任务是将每个图分类为两类中的一类
-  通过检查数据集的第一个图对象，我们可以看到它有 17 个节点（具有 7 维特征向量）和 38条边（平均节点度为2.24`），它还有一个标签 (y=[1])。
-  除了之前的数据集之外，还提供了额外的 4 维边缘特征 (edge_attr=[38, 4])。 然而，为了简单起见，我们这次不会使用它们。

-  PyTorch Geometric 提供了一些有用的实用程序来处理图数据集，例如，我们可以打乱数据集并使用前 150 个图作为训练图，同时使用剩余的图形进行测试

In [6]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:150]
test_dataset = dataset[150:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 150
Number of test graphs: 38


-  批处理对于图数据比较复杂和麻烦。
-  PyTorch Geometric 选择了一种和常见图像数据集不同的方法来实现多个示例的并行化。 
-  在这里，邻接矩阵以对角方式堆叠（创建一个包含多个孤立子图的巨型图），并且节点和目标特征在节点维度中简单地连接。
-  与其他批处理程序相比，该程序具有一些关键优势：
1. 依赖于消息传递方案的 GNN 算子不需要修改，因为属于不同图的两个节点之间不会交换消息；
2. 由于邻接矩阵以稀疏方式保存，仅保存非零条目（即边），因此不存在计算或内存开销。

- PyTorch Geometric 在 torch_geometric.data.DataLoader 类的帮助下自动将多个图批处理为单个巨型图，我们并不需要手动进行上述的复杂步骤。

In [7]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

-  在这里，我们选择 batch_size 为 64，从而产生 3 个（随机洗牌）小批量，包含所有 2⋅64+22=150 个图。

## 训练 GNN 进行图分类通常遵循一个简单的方法

- 通过执行多轮消息传递来嵌入每个节点。
- 将节点嵌入聚合为统一的图嵌入（读出层）。
- 在图嵌入上训练最终分类器。

- 对于整图分类，我们需要一个**读出层（readout layer）**，但最常见的一种是简单地取节点嵌入的平均值：
$$
x_{out}= \frac{1}{|V|}\sum_{v \in V} x^{(L)}_v
$$

- PyTorch Geometric 通过 `torch_geometric.nn.global_mean_pool` 提供此功能，它接受小批量中所有节点的节点嵌入和分配向量批量，以计算批量中每个图的大小为 `[batch_size, hide_channels]` 的图嵌入。也就是说，我们在这里不需要考虑批大小。


- 将 GNN 应用到图分类任务的最终架构如下所示，并允许**完整的端到端训练**

In [9]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        # 使用 GCNConv 
        # 为了让模型更稳定我们也可以使用带有跳跃链接的 GraphConv
        # from torch_geometric.nn import GraphConv
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)
        
    def forward(self, x, edge_index, batch):
        # 1. 获得节点的嵌入
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. 读出层
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. 应用最后的分类器
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


-  在这里，我们再次使用 GCNConv 和 ReLU(x)=max(x,0) 激活来获得局部节点嵌入，然后再将最终分类器应用到图读出层之上。

- 让我们训练我们的网络几个周期，看看它在训练和测试集上的表现如何

In [10]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # 迭代获得各个批数据
         out = model(data.x, data.edge_index, data.batch)  # 前向传播
         loss = criterion(out, data.y)  # 计算损失
         loss.backward()  # 反向传播
         optimizer.step()  # 参数更新
         optimizer.zero_grad()  # 梯度清零

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # 迭代获得各个批数据
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # 取最大概率的类作为预测
         correct += int((pred == data.y).sum())  # 与真实标签做比较
     return correct / len(loader.dataset)  # 计算准确率


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')


<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 002, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 003, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 004, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 005, Train Acc: 0.6467, Test Acc: 0.7368
Epoch: 006, Train Acc: 0.6533, Test Acc: 0.7368
Epoch: 007, Train Acc: 0.7467, Test Acc: 0.7632
Epoch: 008, Train Acc: 0.7267, Test Acc: 0.7632
Epoch: 009, Train Acc: 0.7200, Test Acc: 0.7632
Epoch: 010, Train Acc: 0.7133, Test Acc: 0.7895
Epoch: 011, Train Acc: 0.7200, Test Acc: 0.7632
Epoch: 012, Train Acc: 0.7200, Test Acc: 0.7895
Epoch: 013, Train Acc: 0.7200, Test Acc: 0.7895
Epoch: 014, Train Acc: 0.7133, Test Acc: 0.8421
Epoch: 015, Train Acc: 0.7133, Test Acc: 0.8421
Epoch: 016, Train Acc: 0.7533, Test Acc: 0.7368
Epoch: 017, Train Acc: 0.7400, Test Acc: 0.7632
Epoch: 018, Train Acc: 0.7133, Test Acc: 0.8421
Epoch: 019, Train Acc: 0.7400, Test Acc: 0.7895
Epoch: 020, Train Acc: 0.7533, Test Acc: 0.7368
Epoch: 021, Train Acc: 0.7467, Test Acc:

- 可以看到，我们的模型达到了 76% 左右的测试准确率。 准确率波动的原因可以用相当小的数据集（只有 38 个测试图）来解释，并且一旦将 GNN 应用到更大的数据集，通常就会消失。